# Transformations and Representations



## 1. 3D Data and Open3D

1. Please find mesh files in **data/Q1** folder. Using these mesh files and your own creativity/visualisation, create a "Table" **pointcloud** scene. The table scene should be realistic, scaled appropiately. Use all the meshes given in the folder and treat them as objects kept on the table. 

    You are expected to perform different functions on the individual mesh files: first convert the mesh files to pointclouds and on each pointcloud perform operations such as scaling, rotation, translation. Next, visualize them together. The visualization should represent a pointcloud of a realistic table scene. Save the scene as **.pcd** file. 

    **Please do not copy as we may use your contribution to create a table top dataset.**

    Refer below image for example of a table-top scene:

    <img src="img/1.jpeg"  width="500" >
<br>
<br>

2. Use the final table scene pointcloud obtained from part 1. 
    - Use Open3D to generate partial pointclouds from different camera views (at least 4 views). This means, that you need to crop or capture the points in the pointcloud that are visible only from a given viewpoint. 
    - Using these partial pointclouds, you are now expected to generate the full scene pointcloud back by registering the pointclouds to a global frame. Save the partial and reconstructed pointclouds in different files. 
    - **[ BONUS ]** Finally, compute the error using "Chamfer's Distance (CD)" between the ground truth scene pointcloud and the reconstructed pointcloud. Perform an analysis: 
      1. Why is the CD not 0?
      2. How does the CD change as the number of viewpoints increase / decrease? 
      3. Can we optimize the viewpoints (by hit and trial) such that the CD reduces?

Refer the following link for solving Q1:
- Hidden-Point-Removal Open3D API: http://www.open3d.org/docs/latest/tutorial/Basic/pointcloud.html#Hidden-point-removal

- Chamfer's Distance:  https://pytorch3d.readthedocs.io/en/latest/modules/loss.html



In [ ]:
import open3d as o3d
import numpy as np
import os
from pytorch3d.loss import chamfer_distance
import torch

boat = o3d.io.read_point_cloud("./results/q1/pcdObjects/boat.pcd")
car = o3d.io.read_point_cloud("./results/q1/pcdObjects/car.pcd")
table = o3d.io.read_point_cloud("./results/q1/pcdObjects/table.pcd")
laptop = o3d.io.read_point_cloud("./results/q1/pcdObjects/laptop.pcd")
plane = o3d.io.read_point_cloud("./results/q1/pcdObjects/plane.pcd")
trashcan = o3d.io.read_point_cloud("./results/q1/pcdObjects/trashcan.pcd")

pcds = []
pcds.append(table)

R = laptop.get_rotation_matrix_from_xyz((np.pi/2, np.pi/2, 0))
laptop.rotate(R)
laptop.scale(20, center=laptop.get_center())
laptop.translate((85, 90, 32))
pcds.append(laptop)

R = car.get_rotation_matrix_from_xyz((np.pi/2, -np.pi/4, 0))
car.rotate(R)
car.translate((95, 90, 30))
car.scale(10, center=car.get_center())
pcds.append(car)

R = trashcan.get_rotation_matrix_from_xyz((np.pi/2, 0, 0))
trashcan.rotate(R)
trashcan.translate((98, 106, 33.3))
trashcan.scale(13, center=trashcan.get_center())
pcds.append(trashcan)

R = plane.get_rotation_matrix_from_xyz((np.pi/2, np.pi/4, 0))
plane.rotate(R)
plane.translate((85, 105, 29.5))
plane.scale(30, center=plane.get_center())
pcds.append(plane)

R = boat.get_rotation_matrix_from_xyz((np.pi/2, -np.pi/2, 0))
boat.rotate(R)
boat.translate((88, 75, 30))
boat.scale(20, center=boat.get_center())
pcds.append(boat)

singlePcd = o3d.geometry.PointCloud()
for i in pcds:
    singlePcd += i
# o3d.visualization.draw_geometries([singlePcd])


In [ ]:
newPcds = []
diameter = np.linalg.norm(np.asarray(singlePcd.get_max_bound()) - np.asarray(singlePcd.get_min_bound()))
radius = diameter * 100

camera = [0, 0, diameter]
_, pt_map = singlePcd.hidden_point_removal(camera, radius)
pcd1 = singlePcd.select_by_index(pt_map)
# o3d.visualization.draw_geometries([pcd1])
newPcds.append(pcd1)

camera = [0, diameter, 0]
_, pt_map = singlePcd.hidden_point_removal(camera, radius)
pcd2 = singlePcd.select_by_index(pt_map)
# o3d.visualization.draw_geometries([pcd2])
newPcds.append(pcd2)

camera = [diameter, 0, 3*diameter]
_, pt_map = singlePcd.hidden_point_removal(camera, radius)
pcd3 = singlePcd.select_by_index(pt_map)
# o3d.visualization.draw_geometries([pcd3])
newPcds.append(pcd3)

camera = [diameter, 0, diameter]
_, pt_map = singlePcd.hidden_point_removal(camera, radius)
pcd4 = singlePcd.select_by_index(pt_map)
# o3d.visualization.draw_geometries([pcd4])
newPcds.append(pcd4)

camera = [diameter, 2*diameter, diameter]
_, pt_map = singlePcd.hidden_point_removal(camera, radius)
pcd5 = singlePcd.select_by_index(pt_map)
# o3d.visualization.draw_geometries([pcd5])
newPcds.append(pcd5)

camera = [diameter, diameter, 0]
_, pt_map = singlePcd.hidden_point_removal(camera, radius)
pcd6 = singlePcd.select_by_index(pt_map)
# o3d.visualization.draw_geometries([pcd6])
newPcds.append(pcd6)

reconstructedPcd = o3d.geometry.PointCloud()
for i in newPcds:
    reconstructedPcd += i

# o3d.visualization.draw_geometries([reconstructedPcd])


In [ ]:
aray = np.asarray(reconstructedPcd.points)
array = np.asarray(aray)
x = torch.FloatTensor(aray)
x.unsqueeze_(0)
# print(x.shape)
aray2 = np.asarray(singlePcd.points)
y = torch.FloatTensor(aray2)
y.unsqueeze_(0)
# print(y.shape)
chamfer_distance(y, x)

1. Chamfers Distance tells us how much the point clouds differ from each other. It's basically a loss function for point clouds. If CD == 0, then it means that the point clouds are identical. The number of points in both the point clouds "reconstructedPcd" and "singlePcd" are different as the reconstructed point cloud is just a merged version of point clouds from different views. It does not completely reconstruct the scene. The scene has holes and is not completed.
2. With increase in viewpoints, the scene gets completed and the size of holes/ number of holes in the reconstructed pcd decrease. Hence, the CD decreases with increases in viewpoints.
3. Yes, we can choose the viewpoints such that the CD decreases. If we are limited to a finite number of viewpoints, then choosing viewpoints such that maximum number of unique points get covered will result in a lower CD.


## 2. Euler Angles, Rotation Matrices, and Quaternions
1. Write a function (do not use inbuilt libraries for this question):
    - that returns a rotation matrix given the angles $\alpha$, $\beta$, and $\gamma$ in radians (X-Y-Z).
    - to convert a rotation matrix to quaternion and vice versa. 

2. What is a Gimbal lock? Suppose an airplane increases its pitch from $0°$ to $90°$. 

    - Let $R_{gmb\beta}$ be the rotation matrix for $\beta=90°$. Find $R_{gmb\beta}$.
    - Consider the point $p = [0, 1, 0]ᵀ $ on the pitched airplane, i.e. the tip of the wing. Does there exist any $α , γ$ such that $p_{new} = R_{gmb\beta}\; p$ for:
      1. $p_{new} = [1, 0, 0]ᵀ $
      2. For some  $p_{new}$ on the XY unit circle?
      3. For some  $p_{new}$ on the YZ unit circle?
      
      Show your work for all three parts and briefly explain your reasoning. Why is $\beta=90°$  a “certain problematic value”?

    <img src="img/2.3.jpeg"  width="500" ><br>
    
    <img src="img/2.1.jpeg"  width="500" ><br>

    <img src="img/2.2.jpeg"  width="500" >
    


In [11]:
import numpy as np
import math

# Function Function that returns rotation matrix given angles alpha, beta and gamma (in radians)
def rotMatrix(gamma, beta, alpha):
    matrix = []
    row1 = []
    row1.append(math.cos(beta) * math.cos(alpha))
    row1.append(math.sin(gamma)*math.sin(beta)*math.cos(alpha) - math.cos(gamma)*math.sin(alpha))
    row1.append(math.cos(gamma)*math.sin(beta)*math.cos(alpha) + math.sin(gamma)*math.sin(alpha))
    matrix.append(row1)
    row2 = []
    row2.append(math.cos(beta) * math.sin(alpha))
    row2.append(math.sin(gamma)*math.sin(beta)*math.sin(alpha) + math.cos(gamma)*math.cos(alpha))
    row2.append(math.cos(gamma)*math.sin(beta)*math.sin(alpha) - math.sin(gamma)*math.cos(alpha))
    matrix.append(row2)
    row3 = []
    row3.append(-math.sin(beta))
    row3.append(math.sin(gamma)*math.cos(beta))
    row3.append(math.cos(gamma)*math.cos(beta))
    matrix.append(row3)
    return matrix

# Function that converts a given quarternion to a rotation matrix
def QTM(qx, qy, qz, qw):
    #  qw is the real part of the quarternion
    matrix = []
    row1 = []
    row1.append(1 - 2 * qy**2 - 2 * qz**2)
    row1.append(2 * qx * qy + 2 * qz * qw)
    row1.append(2 * qx * qz - 2 * qy * qw)
    matrix.append(row1)
    row2 = []
    row2.append(2 * qx * qy - 2 * qz * qw)
    row2.append(1 - 2 * qx**2 - 2 * qz**2)
    row2.append(2 * qy * qz + 2 * qx * qw)
    matrix.append(row2)
    row3 = []
    row3.append(2 * qx * qz + 2 * qy * qw)
    row3.append(2 * qy * qz - 2 * qx * qw)
    row3.append(1 - 2 * qx**2 - 2 * qy**2)
    matrix.append(row3)
    return matrix

# Function that converts a given rotation matrix to a quarternion
def MTQ(m):
    ans = []
    if m[2][2] < 0:
        if m[0][0] > m[1][1]:
            trace = 1 + m[0][0] - m[1][1] - m[2][2]
            ans.append(trace)
            ans.append(m[0][1] + m[1][0])
            ans.append(m[2][0] + m[0][2])
            ans.append(m[1][2] - m[2][1])
        else:
            trace = 1 - m[0][0] + m[1][1] - m[2][2]
            ans.append(m[0][1] + m[1][0])
            ans.append(trace)
            ans.append(m[1][2] + m[2][1])
            ans.append(m[2][0] - m[0][2])
    else:
        if m[0][0] < -m[1][1]:
            trace = 1 - m[0][0] - m[1][1] + m[2][2]
            ans.append(m[0][2] + m[2][0])
            ans.append(m[1][2] + m[2][1])
            ans.append(trace)
            ans.append(m[0][1] - m[1][0])
        else:
            trace = 1 + m[0][0] + m[1][1] + m[2][2]
            ans.append(m[1][2] - m[2][1])
            ans.append(m[2][0] - m[0][2])
            ans.append(m[0][1] - m[1][0])
            ans.append(trace)
    ans = np.array(ans)
    ans = ans / math.sqrt(trace) * 2



### Gimbal Lock

- We can achieve the desired orientation using Euler rotations by combining multiple euler transformations. However, the rotations about different axes depend upon each other. We can say that transformations can change the asis of coordinate system. 

- In some situations, two of the axes can line up perfectly. In this case, rotating two of the axes has the same effect. In totality, we are left with only 2 dimensions of movement and we have lost a degree of freedom. This locking type of phenomenon is know as the Gimbal Lock.

### Rotation matrix for airplane increasing pitch by 90 degreees

For $ \beta = \pi / 2$ 

$R_{gmb \beta}$ = $\begin{bmatrix} 1 & 0 & 0 \\ 0 & cos(\gamma) & -sin(\gamma) \\ 0 & sin(\gamma) & cos(\gamma) \end{bmatrix}$ $\begin{bmatrix} 0 & 0 & 1 \\ 0 & 1 & 0 \\ -1 & 0 & 0 \end{bmatrix}$ $\begin{bmatrix} cos(\alpha) & -sin(\alpha) & 0 \\ sin(\alpha) & cos(\alpha) & 0 \\ 0 & 0 & 1 \end{bmatrix}$ 

$R_{gmb \beta}$ = $\begin{bmatrix} 0 & 0 & 1 \\ cos(\gamma)sin(\alpha) + cos(\alpha)sin(\gamma) & cos(\gamma)cos(\alpha) - sin(\alpha)sin(\gamma) & 0 \\ -cos(\gamma)cos(\alpha) + sin(\alpha)sin(\gamma) &  cos(\gamma)sin(\alpha) + cos(\alpha)sin(\gamma) & 0 \end{bmatrix}$



$P_{new} = R_{gmb \beta} * P$ = $\begin{bmatrix} 0 & 0 & 1 \\ cos(\gamma)sin(\alpha) + cos(\alpha)sin(\gamma) & cos(\gamma)cos(\alpha) - sin(\alpha)sin(\gamma) & 0 \\ -cos(\gamma)cos(\alpha) + sin(\alpha)sin(\gamma) &  cos(\gamma)sin(\alpha) + cos(\alpha)sin(\gamma) & 0 \end{bmatrix}$ $\begin{bmatrix} 0 \\ 1  \\ 0  \end{bmatrix}$

= $\begin{bmatrix} 0 \\ cos(\gamma)cos(\alpha) - sin(\alpha)sin(\gamma) \\ cos(\gamma)sin(\alpha) + cos(\alpha)sin(\gamma) \end{bmatrix}$

= $\begin{bmatrix} 0 \\ sin(\alpha + \gamma) \\ cos(\alpha + \gamma) \end{bmatrix}$

1. $P_{new} = [1, 0, 0]ᵀ $ is not possibleas the first entry of $P_{new}$ has to be zero

2. If $P_{new}$ lies on the XY unit cicle, then, it's z coordinate will be zero and the sum of squares of the x and y coordinates will be 1. However, $P_{new}$ has to have x coordinate as 0. Therefore, y coordinate has to be +1 or -1. Therefore $P_{new} = [0, +1, 0]ᵀ$ and $P = [0, -1, 0]ᵀ$ are two posible solutions. Now, if $ cos(\alpha + \gamma) = 0 $ and $ sin(\alpha + \gamma)  = +1 , -1 $, then this gives us infinte solutions for $ \alpha + \gamma $ where their sum is either $\pi/2$ or $-\pi/2$.

3. If $P_{new}$ lies on the YZ unit cicle, then for any y belonging to {0,1}, we can get an $z^2$ = {1 - $y^2$}. Therefore, wehave infiite possibilities for $P_{new}$. Now, $cos(\alpha + \gamma)^2$ + $sin(\alpha + \gamma)^2$ is always 1 which is already satisfied by the equation: $z^2$ = 1 - $y^2$. Therefore, we will be able to get $ \alpha,  \gamma $ for all these solutions. 

- The value of 90 degrees as $ \beta $ is a problematic value because it results in the issue of gimbal lock. One dimension of movement gets inhibited and hence is an issue.


## 3. Transformations and Homogeneous Coordinates

1. Watch this [video](https://www.youtube.com/watch?v=PvEl63t-opM) to briefly understand homogeneous coordinates. 
    1. What are points at infinity? What type of transformation can you apply to transform a point from infinity to a point that is not at infinity? 
    2. Find the vanishing point for the given images in the **data/Q3** folder. Complete function **FilterLines()** and  **GetVanishingPoint()** in the given starter code.

<br>

2. Using homogeneous coordinates we can represent different types of transformation as point transforms vs. frame transforms. Concatenation of transforms (whether you post multiply transformation matrices or pre-multiply transformation matrices) depends on the problem and how you are viewing it. Try to understand the difference between frame vs. point transformations from this [video](https://youtu.be/Za7Sdegf8m8?t=1834). We have 5 camera frames A, B, C, D and E. Given *frame* transformation $A \rightarrow B$ ,  $B \rightarrow C$ ,  $D \rightarrow C$ ,  $D \rightarrow E$. Compute *frame transformation*  $D \rightarrow E$. Also, given the co-ordinates of a point *x* in *D's* frame, what transformation is required to get *x's*  co-ordinates in *E's* frame? 

    <img src="img/3.jpeg"  width="500" >



### What are points at infinity? 

- In geometry, is is considered to be an ideal point or an idealizing limit used to depict the end of each line.

- In projective geometry, a pair of lines can be said to determine a  point similar to the fact that a pair of points can determine a line. It is said to be the point which is at the end of two parallel lines or basically the point which is supposed to be the intersection of these two parallels.      

- In homogeneous coordinates, it represents a point that gives us a sense of direction but is infinitely far away in that direction.

### What type of transformation can you apply to transform a point from infinity to a point that is not at infinity

- When we represent a vector in a homogeneous coordinate system, we add another dimension to the vector in the matrix representation. If we have a vector `<x, y, z>` in a 3 dimensional space, we can represent it as `[x', y', z', w]'` where w represents the added dimension. Now, `x = x'/w, y = y'/w and z = z'/w` are the corresponding relations. Therefore, if we have `w = 1`, we can easily map homogeneous coordinates to our original coordiante system.

- Wer can apply these homogeneous transformations to represent points at infinity. In this case, the value of w will be 0. And hence x, y, z all will be values obtained after dividing by zero leading to the tendency of infinity.

In [2]:
import os
import cv2
import math
import numpy as np
import matplotlib.pyplot as plt

def ReadImage(InputImagePath):
    Images = []                     # Input Images will be stored in this list.
    ImageNames = []                 # Names of input images will be stored in this list.
    
    # Checking if path is of file or folder.
    if os.path.isfile(InputImagePath):						    # If path is of file.
        InputImage = cv2.imread(InputImagePath)                 # Reading the image.
        
        # Checking if image is read.
        if InputImage is None:
            print("Image not read. Provide a correct path")
            exit()
        
        Images.append(InputImage)                               # Storing the image.
        ImageNames.append(os.path.basename(InputImagePath))     # Storing the image's name.

	# If path is of a folder contaning images.
    elif os.path.isdir(InputImagePath):
		# Getting all image's name present inside the folder.
        for ImageName in os.listdir(InputImagePath):
			# Reading images one by one.
            InputImage = cv2.imread(InputImagePath + "/" + ImageName)
			
            Images.append(InputImage)							# Storing images.
            ImageNames.append(ImageName)                        # Storing image's names.
        
    # If it is neither file nor folder(Invalid Path).
    else:
        print("\nEnter valid Image Path.\n")
        exit()

    return Images, ImageNames
        
def GetLines(Image):
    # Converting to grayscale
    GrayImage = cv2.cvtColor(Image, cv2.COLOR_BGR2GRAY)
    # Blurring image to reduce noise.
    BlurGrayImage = cv2.GaussianBlur(GrayImage, (5, 5), 1)
    # Generating Edge image
    EdgeImage = cv2.Canny(BlurGrayImage, 40, 255)

    # Finding Lines in the image
    Lines = cv2.HoughLinesP(EdgeImage, 1, np.pi / 180, 50, 10, 15)

    # Check if lines found and exit if not.
    if Lines is None:
        print("Not enough lines found in the image for Vanishing Point detection.")
        exit(0)
    
    return Lines
    
def FilterLines(Lines):
    output = []
    maxLines = 100 # Maximum number of lines to be considered.
    critAngle = 5 # To Avoid Taking lines close to 0 or 90 degrees
    maxSlope = 1e5 # To represent Perpendicular lines.

    for everyLine in Lines:    
        # Getting the end points of the line.
        [x1, y1, x2, y2] = everyLine[0]   
        
        Slope = maxSlope
        if x1 != x2:
            Slope = (y2 - y1) / (x2 - x1)
            
        yIntercept = y2 - (Slope * x2)
        # y = mx + yIntercept
        
        angle = abs(math.degrees(math.atan(Slope)))
        if angle >= critAngle: # Avoid Horizontal lines.
            if angle <= 90 - critAngle: # Avoid Vertical lines.
                lengthLine = math.sqrt((y2 - y1) * (y2 - y1) + (x2 - x1) * (x2 - x1))
                output.append([Slope, yIntercept, lengthLine, x1, y1, x2, y2])
    
    # Sorting the lines based on length and taking the best few lines out of all.
    output = sorted(output, key=lambda x: x[2])
    if(len(output) > maxLines):
        output = output[:maxLines]

    return output 

def GetVanishingPoint(lines):
    err = 1e5 # To store minimum error across all points.
    ans = None 
    
    for i in range(len(lines)):
        for j in range(i + 1, len(lines)):
            slope1, intercept1 = lines[i][0], lines[i][1]
            slope2, intercept2 = lines[j][0], lines[j][1]
            if slope1 != slope2: # Avoiding parallel lines.
                pointError = 0
                xIntersect = (intercept2 - intercept1) / (slope1 - slope2)
                yIntersect = slope1 * xIntersect + intercept1
                for k in range(len(lines)): # Calculating error by iterating over all lines.
                    slope3, intercept3 = lines[k][0], lines[k][1]
                    perpSlope = -1 / slope3
                    newIntercept = yIntersect - perpSlope * xIntersect
                    xInt = (intercept3 - newIntercept) / (perpSlope - slope3) # X Intersection with perpendicular line.
                    yInt = perpSlope * xInt + newIntercept # Y Intersection with perpendicular line.
                    # Distance between the two points of intersection.
                    leng = math.sqrt((xInt - xIntersect) * (xInt - xIntersect)  + (yInt - yIntersect) * (yInt - yIntersect))
                    pointError += leng*leng
                pointError = math.sqrt(pointError)
                if pointError < err:  # Chek if new error is less than current minimum
                    err = pointError
                    ans = [xIntersect, yIntersect] # If error is minimum, the new point is the answer.
    return ans

Images, ImageNames = ReadImage("./data/Q3/")            # Reading all input images


for i in range(len(Images)):
    Image = Images[i]

    # Getting the lines form the image
    Lines = GetLines(Image)

    FilteredLines = FilterLines(Lines)
    # Get vanishing point
    VanishingPoint = GetVanishingPoint(FilteredLines)

    # Checking if vanishing point found
    if VanishingPoint is None:
        print("Vanishing Point not found. Possible reason is that not enough lines are found in the image for determination of vanishing point.")
        continue

    # Drawing lines and vanishing point
    Lines = FilteredLines

    for Line in Lines:
        cv2.line(Image, (Line[3], Line[4]), (Line[5], Line[6]), (0, 255, 0), 2)
    cv2.circle(Image, (int(VanishingPoint[0]), int(VanishingPoint[1])), 10, (0, 0, 255), -1)

    # Showing the final image
    cv2.imshow("OutputImage", Image)
    cv2.waitKey(0)

### Frame Transformations

Let us consider $P^{A}$ to be the position vector of $\vec P$ in frame A.

Let $T_{A}^{B}$ be the homogeneous transformation matrix of frame A relative to B. Hence, multiplying a vetor in frame B with this matrix would yield us the mapping of the point in frame A. Therefore, $P^{B}$ = $T_{A}^{B}P^{A}$ (The notation which I have used is different from the one given in the question)


Therefore we are given: $T_{A}^{B}$, $T_{B}^{C}$, $T_{D}^{C}$ and $T_{A}^{E}$.

Now we want to obtain : $T_{D}^{E}$

We can use the statement: $P^{E}$ = $T_{D}^{E}P^{D}$ 

Now, $P^{E}$ = $T_{A}^{E}P^{A}$
Also, $P^{A}$ = $T_{B}^{A}P^{B}$. But we have $T_{A}^{B}$ which is nothing but the inverse of $T_{B}^{A}$. 

So, $T_{B}^{A}$ = $(T_{A}^{B})^{-1}$  
Therefore, Now, $P^{E}$ = $T_{A}^{E}P^{A}$ = $T_{A}^{E} (T_{A}^{B})^{-1} P^{B} $

Now, $P^{B}$ = $T_{C}^{B}P^{C}$
Also, $P^{C}$ = $T_{B}^{C}P^{B}$. But we have $T_{B}^{C}$ which is nothing but the inverse of $T_{C}^{B}$. 

So, $T_{C}^{B}$ = $(T_{B}^{C})^{-1}$  
Therefore, Now, $P^{E}$ = $T_{A}^{E} (T_{A}^{B})^{-1} P^{B} $ = $T_{A}^{E} (T_{A}^{B})^{-1} (T_{B}^{C})^{-1} P^{C} $

Now, $P^{C}$ = $T_{D}^{C}P^{D}$

Therefore, Now, $P^{E}$ = $T_{A}^{E} (T_{A}^{B})^{-1} (T_{B}^{C})^{-1} P^{C}$ = $T_{A}^{E} (T_{A}^{B})^{-1} (T_{B}^{C})^{-1} T_{D}^{C}P^{D} $

Now comparing with the original statement:

$T_{D}^{E} = T_{A}^{E} (T_{A}^{B})^{-1} (T_{B}^{C})^{-1} T_{D}^{C}$


Now, the last part of the question also asks us to find $T_{E}^{D}$ whih will be nothing but the inverse of $T_{D}^{E}$.

## 4. LiDAR and Registration

Point clouds are a collection of points that represent a 3D shape or feature. Each point has its own set of X, Y and Z coordinates and in some cases additional attributes. A popular way to obtain this is by photogrammetry, though here we will use LiDAR data.

LiDAR is a remote sensing process which collects measurements used to create 3D models and maps of objects and environments. Using ultraviolet, visible, or near-infrared light, LiDAR gauges spatial relationships and shapes by measuring the time it takes for signals to bounce off objects and return to the scanner.

Download the data from [here](https://iiitaphyd-my.sharepoint.com/:f:/g/personal/venkata_surya_students_iiit_ac_in/EnYAMaTVIhJItzKYqtahE30BRKB6p6UfHN3TyJzvo6Mw0g?e=PegWds). It contains the LIDAR sensor output and odometry information per frame.

  The .bin files contain the 3D point cloud captured by the LIDAR in this format - x, y, z, and reflectance. 

  The odometry information is given in the `odometry.txt` file, which is a 12 element vector. Reshape each of the first 77 rows to a 3x4 matrix to obtain the pose.
    
The point cloud obtained is with respect to the LiDAR frame. The poses however, are in the camera frame. If we want to combine the point clouds from various frames, we need to bring them to the camera frame. 

1. Refer to the image below and apply the required transformation to the point cloud. 
<br>

    <img src="img/4.jpeg"  width="500" >

<br>

2. Then, register all point clouds into a common reference frame and visualise it (Open3D). It is helpful to use homogeneous coordinates to keep track of the different frames.

3. Write a function to transform the registered point cloud from the world to the $i^{th}$ camera frame, wherein $i$ is the input to the function.




In [4]:
import os
import numpy as np
import open3d as o3d
import copy

storeData = []
nameFile = "./data/Q4/LiDAR/0000"
for i in range(10, 87):
    x = nameFile + str(i) + ".bin"
    binPcd = np.fromfile(x, dtype=np.float32)
    binPcd = binPcd.reshape((-1, 4))[:, 0:3]
    storeData.append(binPcd)

storeData = np.array(storeData, dtype=object)

odoFile = "./data/Q4/odometry.txt"
filePtr = open(odoFile, "r")
matrix = []
for i in range(77):
    row = (filePtr.readline()).split()
    rowInMatrix= np.array(list(map(float, row)))
    rowInMatrix = np.reshape(rowInMatrix, (3, 4))
    matrix.append(rowInMatrix)

lidToCam = np.array([[0,0,1,0],[-1,0,0,0], [0,-1,0,0],[0,0,0,1]])
lidToCam = np.transpose(lidToCam)

allPoses = []
for i in range(77):
    numPoints = storeData[i].shape[0]
    currPose = []
    for j in range(numPoints):
        homoVec = np.array([storeData[i][j][0], storeData[i][j][1], storeData[i][j][2], 1])
        convertedVec = np.matmul(lidToCam, homoVec)
        currPose.append(convertedVec)
    allPoses.append(currPose)


In [5]:
homoMatrix = []
for i in range(77):
    curr = np.array([[matrix[i][0][0], matrix[i][0][1], matrix[i][0][2], matrix[i][0][3]], [matrix[i][1][0], matrix[i][1][1], matrix[i][1][2], matrix[i][1][3]], [matrix[i][2][0], matrix[i][2][1], matrix[i][2][2], matrix[i][2][3]], [0,0,0,1]]) 
    homoMatrix.append(curr)
worldFrame = np.array([[0,0,1,0],[-1,0,0,0], [0,-1,0,0] ,[0,0,0,1]])
ctw = []
wtc = []
for i in range(77):
    ctw.append(np.matmul(worldFrame, homoMatrix[i]))
    wtc.append(np.linalg.inv(ctw[i]))

inCommonFrame = []
for i in range(77):
    allPoses[i] = np.array(allPoses[i])
    extent = allPoses[i].shape[0]
    for j in range(extent):
        inCommonFrame.append(np.matmul(homoMatrix[i], allPoses[i][j])[0:3])

pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(inCommonFrame)
# o3d.visualization.draw_geometries([pcd])

In [6]:
def convertCamera(x, pcd):
    o3d.visualization.draw_geometries([pcd.transform(wtc[x])])
